In [2]:
!pip install opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [1]:
!pip install mediapipe

  Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached jax-0.5.3-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.5.3-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-win_amd64.whl.metadata (8.3 kB)
Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl (51.0 MB)
Using cached sounddevice-0.5.1-py3-none-win_amd64.whl (363 kB)
Using cached jax-0.5.3-py3-none-any.whl (2.4 MB)
Using cached jaxlib-0.5.3-cp312-cp312-win_amd64.whl (65.8 MB)
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ----------------------------

In [4]:
!pip install pygame

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.6 MB 730.2 kB/s eta 0:00:14
   - -------------------------------------- 0.5/10.6 MB 730.2 kB/s eta 0:00:14
   -- ------------------------------------- 0.8/10.6 MB 819.2 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/10.6 MB 1.1 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/10.6 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/10.6 MB 1.3 MB/s eta 0:00:07
   ------ --------------------------------- 1.8/10.6 MB 1.3 MB/s eta 0:00:07
   ------ --------------------------------- 1.8/10.6 MB 1.3 MB/s eta 0:00:07
   ------- -------------------------------- 2.1/10.6 MB 924.9 kB/s eta 0:00:10
   ------- -------------------------------- 2.1/10.6 MB 924.9 kB/s eta 0:00:10
   -------

In [10]:
import cv2
import mediapipe as mp
import numpy as np
import pygame
import time
from collections import deque

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    refine_landmarks=True 
)


LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]


HEAD_POSE_LANDMARKS = [1, 168, 6, 197, 195, 5, 4]


pygame.mixer.init()
pygame.mixer.music.load("sms.mp3")  

EAR_HISTORY = deque(maxlen=10)  
EAR_SMOOTHING_FACTOR = 0.2  

DYNAMIC_EAR_THRESHOLD = 0.25  
MIN_EAR_THRESHOLD = 0.18  

DISTRACTION_TIME = 2.0  

def calculate_ear(landmarks, eye_points):
    """Improved EAR calculation with better stability."""
 
    horizontal_dist = np.linalg.norm(landmarks[eye_points[0]] - landmarks[eye_points[3]])
    
    vertical_dist1 = np.linalg.norm(landmarks[eye_points[1]] - landmarks[eye_points[5]])
    vertical_dist2 = np.linalg.norm(landmarks[eye_points[2]] - landmarks[eye_points[4]])
    avg_vertical_dist = (vertical_dist1 + vertical_dist2) / 2.0
    ear = avg_vertical_dist / (horizontal_dist + 1e-6)  
    return ear

def get_head_pose(landmarks, frame_shape):
    """Estimates head pose (simplified)."""
    nose = landmarks[1]
    chin = landmarks[152]
    left_eye = landmarks[33]
    right_eye = landmarks[263]
    
    
    horizontal_ratio = (right_eye[0] - left_eye[0]) / frame_shape[1]
    vertical_ratio = (chin[1] - nose[1]) / frame_shape[0]
    
    
    is_looking_away = (horizontal_ratio < 0.2) or (horizontal_ratio > 0.8)
    is_looking_down = (vertical_ratio > 0.5)  
    
    return is_looking_away or is_looking_down

def update_dynamic_threshold(current_ear):
    """Adjusts EAR threshold based on user's normal eye state."""
    global DYNAMIC_EAR_THRESHOLD
    if len(EAR_HISTORY) > 5:  
        avg_ear = np.mean(EAR_HISTORY)
        DYNAMIC_EAR_THRESHOLD = max(MIN_EAR_THRESHOLD, avg_ear * 0.7)  

cap = cv2.VideoCapture(0)

distracted_start = None
last_alert_time = 0
ALERT_COOLDOWN = 5  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = np.array([(int(pt.x * frame.shape[1]), int(pt.y * frame.shape[0])) 
                          for pt in face_landmarks.landmark])
            
            
            left_ear = calculate_ear(landmarks, LEFT_EYE)
            right_ear = calculate_ear(landmarks, RIGHT_EYE)
            avg_ear = (left_ear + right_ear) / 2.0
            EAR_HISTORY.append(avg_ear)
            
            
            update_dynamic_threshold(avg_ear)
            
           
            is_looking_away = get_head_pose(landmarks, frame.shape[:2])
            
           
            if (avg_ear < DYNAMIC_EAR_THRESHOLD) or is_looking_away:
                if distracted_start is None:
                    distracted_start = time.time()
                elif time.time() - distracted_start > DISTRACTION_TIME:
                    current_time = time.time()
                    if current_time - last_alert_time > ALERT_COOLDOWN:
                        cv2.putText(frame, "⚠ DISTRACTED!", (50, 50), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        pygame.mixer.music.play()
                        last_alert_time = current_time
            else:
                distracted_start = None  

   
    cv2.putText(frame, f"EAR: {avg_ear:.2f} (Thresh: {DYNAMIC_EAR_THRESHOLD:.2f})", 
               (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    
    cv2.imshow("Improved Distraction Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()